# TinyMS ResNet50 教程

### 在本教程中，我们会演示获取ResNet50 ckpt文件，使用TinyMS API启动推理服务器和进行推理的过程。

## 环境要求
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.1`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## 介绍

TinyMS是一个高级API，目的是让新手用户能够更加轻松地上手深度学习。TinyMS可以有效地减少用户在构建、训练、验证和推理一个模型过程中的操作次数。TinyMS也提供了教程和文档帮助开发者更好的上手和开发。

本教程中，由于使用CPU训练ResNet50模型过于耗时，所以本教程将直接提供训练好的ResNet50 ckpt文件。步骤包含4部分：获取ckpt文件、定义servable json，启动服务器和推理，其中服务器在子进程中启动。


## 步骤

### 1. 获取ckpt文件

本教程提供两个由不同训练集训练而成的ResNet50模型，一个是由[ImageNet2012](http://www.image-net.org/challenges/LSVRC/2012/)数据集训练得来，另一个是由[cifar10](http://www.cs.toronto.edu/~kriz/cifar.html)数据集训练得来。启动服务器的前提条件是需要ResNet50 ckpt文件，可以点击[resnet_imagenet](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com:443/resnet-50/ckpt_files/imagenet2012/resnet50.ckpt?AccessKeyId=HUXAJ6GDDQV6E9JLVZRG&Expires=1645083010&Signature=UmcUjCyDs/0jPRJCWu2e8xS5ooc%3D)下载ResNet50_imagenet ckpt文件，或者点击[resnet_cifar](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com:443/resnet-50/ckpt_files/cifar10/resnet50.ckpt?AccessKeyId=HUXAJ6GDDQV6E9JLVZRG&Expires=1645083487&Signature=XUPGdTwctaw%2Bep04W3Bo4qxQYG0%3D)下载ResNet50_cifar ckpt文件


将ckpt文件保存到`/etc/tinyms/serving/resnet50_<dataset_name>/resnet50.ckpt`

### 2. 定义servable.json

定义servable json文件，servable json文件定义了servable名称，模型名称，模型格式和分类数量，以便后续推理使用。在该步骤中，将演示推理`ResNet50_imagenet2012`模型

In [ ]:
import os
import json
import subprocess
from tinyms.serving import start_server, predict, list_servables


servable_json = [{'name': 'resnet50_imagenet2012', 
                  'description': 'This servable hosts a resnet50 model predicting mushrooms', 
                  'model': {
                      "name": "resnet50", 
                      "format": "ckpt", 
                      "class_num": 9}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

如果用户想要运行`ResNet50_cifar10`模型，将上述代码中的`servable_json`字段替换成如下代码段：
```
servable_json = [{'name': 'resnet50_cifar10', 
                  'description': 'This servable hosts a resnet50 model predicting 10 classes of objects', 
                  'model': {
                      "name": "resnet50", 
                      "format": "ckpt", 
                      "class_num": 10}}]
```
并且请确保在修改servable后重新运行代码。

### 3. 启动服务器

#### 3.1 介绍
TinyMS推理是C/S（Client/Server）架构。TinyMS使用[Flask](https://flask.palletsprojects.com/en/1.1.x/)这个轻量化的网页服务器架构作为C/S通讯的基础架构。为了能够对模型进行推理，用户必须首先启动服务器。如果成功启动，服务器会在子进程中运行并且会监听从地址127.0.0.1，端口号5000发送来的POST请求并且使用MindSpore作为后端来处理这些请求。后端会构建模型，运行推理并且返回结果给客户端

#### 3.2 启动服务器
运行下列代码以启动服务器：

In [ ]:
cmd = ['/bin/bash', '-C', 'python', 'from tinyms.serving import start_server', 'start_server()']
server_process = subprocess.Popen(cmd,stdout=subprocess.PIPE,shell=True)

### 4. 推理

#### 4.1 上传图片

`ResNet50_imagenet2012`模型需要用户上传一张蘑菇图片作为输入，而`ResNet50_cifar10`模型需要用户上传一张属于如下10个类别的图片作为输入：`['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']`。点击[蘑菇](http://5b0988e595225.cdn.sohucs.com/images/20180615/cb4b7f551e6e4bbaa99ae7e4c0e6f6a3.jpeg)下载本教程中使用的蘑菇图片以运行`ResNet50_imagenet2012`，或者点击[飞机](https://www.in-vendita.it/sh-img/US_Navy_051105-F-5480T-005_An_F-14D_Tomcat_conducts_a_mission_over_the_Persian_Gulf-region_f14%2Btomcat.jpg)以运行`ResNet50_cifar10`。上传图片，如果使用命令行终端，可以使用'scp'或者'wget'获取图片，如果使用Jupyter，点击菜单右上方的'Upload'按钮并且选择上传的图片。

将图片保存在根目录下，重命名为'mushroom.jpeg'或者'airplane.jpg'

#### 4.2 List servables

使用`list_servables`函数检查当前后端的serving模型

In [ ]:
list_servables()

如果输出的`description`字段显示这是一个`resnet50_imagenet2012`的模型，则可以继续到下一步发送推理请求，否则如果是`resnet50_cifar10`，需要将接下来的`predict`代码段替换成代码块下方教程处的代码

#### 4.3 发送推理请求

运行`predict`函数发送推理请求，第4个参数选择`TOP1_CLASS`或`TOP5_CLASS`以指定输出策略:

In [ ]:
# predict(image_path, servable_name, dataset, topk_strategy)
# predict方法的4个参数分别是图片路径、servable名称，数据集名称（默认MNIST，此处需手动指定）和输出策略
predict("/root/mushroom.jpeg", "resnet50_imagenet2012", "imagenet2012", "TOP1_CLASS")

如果用户想要运行`ResNet50_cifar10`， 将上方代码替换成如下：
```
predict("/root/airplane.jpg", "resnet50_cifar10", "cifar10", "TOP1_CLASS")
```

## 检查输出

如果用户运行`ResNet50_imagenet2012`且能看到类似如下输出:  
```
{'prediction: ': 'Amanita毒蝇伞,伞菌目,鹅膏菌科,鹅膏菌属,主要分布于我国黑龙江、吉林、四川、西藏、云南等地,有毒', 'score': '0.99119007587432861328'}
```
那么意味着已经进行了一次成功的推理

如果用户运行`ResNet50_cifar10`，输出应该类似于：
```
{'prediction: ': 'airplane', 'score': '0.99997282028198242188'}
```

## 关闭服务器

重启服务器并使用另一种模型，点击左上方`Kernel`，再点击`Restart & Clear Output`，并替换`servable_json`代码段

关闭服务器，如果使用终端，可以直接CTRL + C关闭，如果使用Jupyter，点击上方`Kernel`再点击`Shutdown`